# Percolation Thesis

## Preprocessing

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# from numba import cuda

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

print("Tensorflow", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
def import_data(name='HybridEllipsePercolation.txt', sep1=" ", header1=None, shuffle=True):
    data = pd.read_csv(name, sep=sep1, header=header1)
    data.columns = ["r1", "2a2", "r2", "frac", "Nc", "Nc Std. Dev", "eta c" ]
    # data.reset_index(inplace=True)
    
    if shuffle:
        data = data.sample(frac=1).reset_index(drop=True)
    return data

In [ ]:
rawdata = import_data(shuffle=True)
dataset = rawdata.copy()
dataset.pop("Nc Std. Dev")
dataset.tail()

In [ ]:
# check for missing values
dataset.isna().sum()
# drop missing values
dataset = dataset.dropna()

In [ ]:
# split into training and test sets

train_dataset = dataset.sample(frac=1.0, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
sns.pairplot(train_dataset[['r1', '2a2', 'r2', 'frac', 'Nc', 'eta c']], diag_kind='kde')

In [ ]:
train_dataset.describe().transpose()

In [ ]:
train_dataset.dtypes

In [ ]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('eta c')
test_labeles = test_features.pop('eta c')

In [ ]:
# (adaptively) normalize the data using keras 

normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(train_features))

In [ ]:
train_dataset[:1]

In [ ]:
train_labels[:1]

In [ ]:
first = np.array(train_features[:1])
with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', normalizer(first).numpy())

In [ ]:
normalized_train_features = normalizer(np.array(train_features.copy())).numpy()

## Design an MLP for the normalized Data
- 5 inputs, 1 output
- inputs are -> normalized_train_features
- momentum = 0.1
- random initial weights
- 2 layers
- dynamically change the number of neurons in each layer, starting with 5-5-3-1

### Tensorflow MLP Regression

In [ ]:
def add_bias(data):
    N1 = np.shape(data)[0]
    N2 = np.shape(data)[1]
    a = -1*np.ones((N1,N2+1))
    a[:,:-1] = data
    return a

In [ ]:
# Load data
X = add_bias(normalized_train_features)
Y = train_labels
# Set the input shape
input_shape = (6,)
print(f'Feature shape: {input_shape}')

# Create the model
model = Sequential()
model.add(Dense(16, input_shape=input_shape, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))

# Configure the model and start training
model.compile(loss='mean_absolute_error', optimizer='SGD', metrics=['mean_squared_error'])
model.fit(X, Y, epochs=100, batch_size=200, verbose=1, validation_split=0.2)

In [ ]:
model.summary()

### My Own MLP

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

# from numba import cuda

class MLP_Regression:
    def __init__(self, features, labels, training=True):
        """ Initialises MLP Regression Model. 
            """
        self.features = features
        self.labels = labels

    
    def set_architecture(self, number_of_hidden_layers, nodes_in_hidden_layers=[5,3]):
        """ Default is TRAINING with 5-5-3-1 architecture.
            Number of hidden layers and length of nodes list must match. """
            
        self.no_hid_layers = number_of_hidden_layers
        self.nodes_per_layer = nodes_in_hidden_layers

        if len(nodes_in_hidden_layers) != number_of_hidden_layers:
            print("Number of layers and nodes in hidden layers do not match.") 


In [ ]:
 if len([5,3,3]) != 2:
     print("here")